#### Download  Dataset From Kaggle and Import Libraries

In [ ]:
# download gtsrb traffic sign dataset from kaggle
!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign

In [ ]:
# create dataset directory and unzip the downloaded dataset
!mkdir dataset
!unzip -q gtsrb-german-traffic-sign.zip -d /kaggle/working/dataset

In [ ]:
# import all neccessary modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.optimizers import Adam, SGD, RMSprop,Adagrad
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization,Input

#### Preprocessing Image ####

In [ ]:
images = []
labels = []
classes = 43
for i in range(classes):
    path = '/kaggle/working/dataset/train/'+ str(i)
    raw_images = os.listdir(path)
    for img in raw_images:
        try:
            image = Image.open(path + '/'+ img)
            image = image.resize((50,50))
            image = np.array(image)
            images.append(image)
            labels.append(i)
        except Exception as e:
            print(e)

#converting to numpy array
images = np.array(images)
labels = np.array(labels)

In [ ]:
classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)',
            2:'Speed limit (50km/h)',
            3:'Speed limit (60km/h)',
            4:'Speed limit (70km/h)',
            5:'Speed limit (80km/h)',
            6:'End of speed limit (80km/h)',
            7:'Speed limit (100km/h)',
            8:'Speed limit (120km/h)',
            9:'No passing',
            10:'No passing veh over 3.5 tons',
            11:'Right-of-way at intersection',
            12:'Priority road',
            13:'Yield',
            14:'Stop',
            15:'No vehicles',
            16:'Veh > 3.5 tons prohibited',
            17:'No entry',
            18:'General caution',
            19:'Dangerous curve left',
            20:'Dangerous curve right',
            21:'Double curve',
            22:'Bumpy road',
            23:'Slippery road',
            24:'Road narrows on the right',
            25:'Road work',
            26:'Traffic signals',
            27:'Pedestrians',
            28:'Children crossing',
            29:'Bicycles crossing',
            30:'Beware of ice/snow',
            31:'Wild animals crossing',
            32:'End speed + passing limits',
            33:'Turn right ahead',
            34:'Turn left ahead',
            35:'Ahead only',
            36:'Go straight or right',
            37:'Go straight or left',
            38:'Keep right',
            39:'Keep left',
            40:'Roundabout mandatory',
            41:'End of no passing',
            42:'End no passing veh > 3.5 tons' }

In [ ]:
print('Shape of Image Data: ' + str(images.shape))
print('Shape of Label Data: ' + str(labels.shape))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size = 0.25, random_state = 42, shuffle=True)
print("X_train.shape", x_train.shape)
print("X_valid.shape", x_test.shape)
print("y_train.shape", y_train.shape)
print("y_valid.shape", y_test.shape)

In [ ]:
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

#### Create Model Architecture ####


In [ ]:
def create_model():
  model = Sequential([
        Input(shape=(50, 50, 3)),
        Conv2D(filters=16, kernel_size=(8, 8), activation='relu'),
        Conv2D(filters=32, kernel_size=(8, 8), activation='relu'),
        MaxPool2D(pool_size=(2, 2)),
        BatchNormalization(axis=-1),
        Dropout(rate=0.5),
        Conv2D(filters=64, kernel_size=(4, 4), activation='relu'),
        Conv2D(filters=128, kernel_size=(4, 4), activation='relu'),
        MaxPool2D(pool_size=(2, 2)),
        BatchNormalization(axis=-1),
        Dropout(rate=0.5),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(43, activation='softmax')
    ])
  return model

**Creating Model Architecture**

In [ ]:
model=create_model()
model.summary()

#### Create Checkpoints
This is helpfull becuase it saves the learning This approach is beneficial because it saves the model's state after each epoch. This proves invaluable if the training process is interrupted for any reason, as it allows you to resume from the last saved checkpoint rather than starting from scratch

In [ ]:
import glob
import os
# from google.colab import drive
# # Mount Google Drive
# drive.mount('/content/drive')
# Set the checkpoint directory to a folder in Google Drive
checkpoint_dir = "/kaggle/working/checkpoints"
# Only create the checkpoint directory if it does not exist
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
else:
    print("Checkpoint directory already exists.")
# os.makedirs(checkpoint_dir, exist_ok=True)

In [ ]:
def get_latest_checkpoint(name):
    # Pattern to match checkpoint files for the given optimizer name
    checkpoint_files = glob.glob(f'{checkpoint_dir}/{name}_epoch*.weights.h5')
    # If no checkpoint files exist, return None
    if not checkpoint_files:
        return None
    # Find the most recently created checkpoint file
    latest_checkpoint = max(checkpoint_files, key=os.path.getctime)
    return latest_checkpoint

 # Define the checkpoint callback

def save_checkpoint(name):
  print(f"Saving checkpoint for {name} optimizer...")
  checkpoint_callback = ModelCheckpoint(
        filepath=f'{checkpoint_dir}/{name}_epoch{{epoch:02d}}.weights.h5',  # Save each epoch
        save_weights_only=True,
        save_freq='epoch',
        verbose=1
    )
  return checkpoint_callback

In [ ]:
# Hyperparameters
epochs=5

#### Part 1 Optimization Algorithms comparision


*   Gradient Descent (GD/Batch GD)
*   Minibatch SGD
*   Stochastic Gradient Descent


In [ ]:
# List of optimizers to compare
part1_optimizers = {
    'SGD (Batch GD)': SGD(learning_rate=0.01),
    'SGD (Mini-batch)': SGD(learning_rate=0.01),
    'SGD (Stochastic)': SGD(learning_rate=0.01)
}

In [ ]:
# Dictionary to hold accuracy results and histories
part1_accuracy_results = {}
part1_histories = {}

# Train and evaluate the model with each optimizer
for name, optimizer in part1_optimizers.items():
    model = create_model()
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    # Adjust batch size based on optimizer
    if 'Batch GD' in name:
        batch_size = len(x_train)  # Entire dataset
    elif 'Mini-batch' in name:
        batch_size = 32  # Mini-batch size
    else:
        batch_size = 1  # Stochastic (single sample)

    # Load latest checkpoint if it exists
    latest_checkpoint = get_latest_checkpoint(name)
    if latest_checkpoint:
        print(f"Resuming from checkpoint: {latest_checkpoint}")
        model.load_weights(latest_checkpoint)

    print(f"Training with {name} optimizer...")
    # checkpoint callback
    checkpoint_callback = save_checkpoint(name)

    try:
        history = model.fit(
            x_train, y_train,
            validation_split=0.2,
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[checkpoint_callback],
            verbose=1
        )
        accuracy = model.evaluate(x_test, y_test, verbose=0)[1]
        part1_accuracy_results[name] = accuracy
        part1_histories[name] = history.history
        print(f"{name} optimizer accuracy: {accuracy}")
    except Exception as e:
        print(f"Training with {name} optimizer failed due to: {e}")

In [ ]:
# Print the results
print("\nComparison of Optimizers:")
for name, accuracy in part1_accuracy_results.items():
    print(f"{name}: {accuracy}")

part1_best_optimizer = max(part1_accuracy_results, key=part1_accuracy_results.get)
print(f"\n Part 1 Best optimizer: {part1_best_optimizer} with accuracy: {part1_accuracy_results[part1_best_optimizer]}")

# Plotting the training and validation accuracy
plt.figure(figsize=(14, 6))
for name, history in part1_histories.items():
    plt.plot(history['accuracy'], label=f'{name} Train Accuracy')
    plt.plot(history['val_accuracy'], label=f'{name} Validation Accuracy')

plt.title('Training and Validation Accuracy for Different Optimizers')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

#### Part 2 Optimization Algorithms comparision


* Gradient Descent
* Gradient Descent with Momentum
* Gradient Descent with Nesterov Momentum


In [ ]:
part2_optimizers = {
    'Gradient Descent': SGD(learning_rate=0.01),
    'Gradient Descent with Momentum': SGD(learning_rate=0.01, momentum=0.9),
    'Gradient Descent with Nesterov Momentum': SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
    }

In [ ]:
# Dictionary to hold accuracy results and histories
part2_accuracy_results = {}
part2_histories = {}

# Directory to save checkpoints
checkpoint_dir = "./checkpoints"

# Train and evaluate the model with each optimizer
for name, optimizer in part2_optimizers.items():
    model = create_model()
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

     # Load latest checkpoint if it exists
    latest_checkpoint = get_latest_checkpoint(name)
    if latest_checkpoint:
        print(f"Resuming from checkpoint: {latest_checkpoint}")
        model.load_weights(latest_checkpoint)

    print(f"Training with {name} optimizer...")
    # checkpoint callback
    checkpoint_callback = save_checkpoint(name)
    print(f"Training with {name} optimizer...")
    try:
        history = model.fit(
            x_train, y_train,
            validation_split=0.2,
            epochs=epochs,
            batch_size=32,
            callbacks=[checkpoint_callback],
            verbose=1
        )
        accuracy = model.evaluate(x_test, y_test, verbose=0)[1]
        part2_accuracy_results[name] = accuracy
        part2_histories[name] = history.history
        print(f"{name} optimizer accuracy: {accuracy}")
    except Exception as e:
        print(f"Training with {name} optimizer failed due to: {e}")

# Print the results
print("\nComparison of Optimizers:")
for name, accuracy in part2_accuracy_results.items():
    print(f"{name}: {accuracy}")
part2_best_optimizer = max(part2_accuracy_results, key=part2_accuracy_results.get)
print(f"\n Part 1 Best optimizer: {part2_best_optimizer} with accuracy: {part2_accuracy_results[part2_best_optimizer]}")



In [ ]:
# Plotting the training and validation accuracy
plt.figure(figsize=(14, 6))
for name, history in part2_histories.items():
    plt.plot(history['accuracy'], label=f'{name} Train Accuracy')
    plt.plot(history['val_accuracy'], label=f'{name} Validation Accuracy')

plt.title('Training and Validation Accuracy for Different Optimizers')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

#### Part 3 Optimization Algorithms comparision


*  AdaGrad
*  RMsProp
*  Adam


In [ ]:
part3_optimizers = {
    'Adam': Adam(learning_rate=0.001),
    'Adagrad': Adagrad(learning_rate=0.01),
    'RMSprop': RMSprop(learning_rate=0.001)
}

In [ ]:
# Dictionary to hold accuracy results and histories
part3_accuracy_results = {}
part3_histories = {}

# Train and evaluate the model with each optimizer
for name, optimizer in part3_optimizers.items():
    model = create_model()
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    # Load latest checkpoint if it exists
    latest_checkpoint = get_latest_checkpoint(name)
    if latest_checkpoint:
        print(f"Resuming from checkpoint: {latest_checkpoint}")
        model.load_weights(latest_checkpoint)

    print(f"Training with {name} optimizer...")
    # checkpoint callback
    checkpoint_callback = save_checkpoint(name)

    print(f"Training with {name} optimizer...")
    try:
        history = model.fit(
            x_train, y_train,
            validation_split=0.2,
            epochs=epochs,
            batch_size=32,
            callbacks=[checkpoint_callback],
            verbose=1
        )

        accuracy = model.evaluate(x_test, y_test, verbose=0)[1]
        part3_accuracy_results[name] = accuracy
        part3_histories[name] = history.history
        print(f"{name} optimizer accuracy: {accuracy}")
    except Exception as e:
        print(f"Training with {name} optimizer failed due to: {e}")

# Print the results
print("\nComparison of Optimizers:")
for name, accuracy in part3_accuracy_results.items():
    print(f"{name}: {accuracy}")

part3_best_optimizer = max(part3_accuracy_results, key=part3_accuracy_results.get)
print(f"\nBest optimizer: {part3_best_optimizer} with accuracy: {part3_accuracy_results[part3_best_optimizer]}")


In [ ]:
# Plotting the training and validation accuracy
plt.figure(figsize=(14, 6))
for name, history in part3_histories.items():
    plt.plot(history['accuracy'], label=f'{name} Train Accuracy')
    plt.plot(history['val_accuracy'], label=f'{name} Validation Accuracy')

plt.title('Training and Validation Accuracy for Different Optimizers')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Install necessary packages
!pip install pydot
!pip install graphviz
from tensorflow.keras.utils import plot_model
from IPython.display import Image
plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)
# Display the saved image
Image(filename='model_architecture.png')